In [1]:
%matplotlib wx
import superball_multilateration_barconstraints as smb
import numpy as np 
import rospy
import rosbag
from matplotlib import pylab as plt
plt.ion()
from sklearn import linear_model, datasets

In [2]:
topics = ['/bbb2/0x2017_0xd','/bbb2/0x2017_0xe','/bbb2/0x2017_0xf','/bbb2/0x2017_0x10',
          '/bbb2/0x2717_0xd','/bbb2/0x2717_0xe','/bbb2/0x2717_0xf','/bbb2/0x2717_0x10',
          '/bbb4/0x2017_0xd','/bbb4/0x2017_0xe','/bbb4/0x2017_0xf','/bbb4/0x2017_0x10',
          '/bbb4/0x2717_0xd','/bbb4/0x2717_0xe','/bbb4/0x2717_0xf','/bbb4/0x2717_0x10',
          '/bbb6/0x2017_0xd','/bbb6/0x2017_0xe','/bbb6/0x2017_0xf','/bbb6/0x2017_0x10',
          '/bbb6/0x2717_0xd','/bbb6/0x2717_0xe','/bbb6/0x2717_0xf','/bbb6/0x2717_0x10',
          '/bbb8/0x2017_0xd','/bbb8/0x2017_0xe','/bbb8/0x2017_0xf','/bbb8/0x2017_0x10',
          '/bbb8/0x2717_0xd','/bbb8/0x2717_0xe','/bbb8/0x2717_0xf','/bbb8/0x2717_0x10',
          '/bbb10/0x2017_0xd','/bbb10/0x2017_0xe','/bbb10/0x2017_0xf','/bbb10/0x2017_0x10',
          '/bbb10/0x2717_0xd','/bbb10/0x2717_0xe','/bbb10/0x2717_0xf','/bbb10/0x2717_0x10',
          '/bbb12/0x2017_0xd','/bbb12/0x2017_0xe','/bbb12/0x2017_0xf','/bbb12/0x2017_0x10',
          '/bbb12/0x2717_0xd','/bbb12/0x2717_0xe','/bbb12/0x2717_0xf','/bbb12/0x2717_0x10'
         ]
topics_node_to_node = np.array([[2,13],
                               [2,14],
                               [2,15],
                               [2,16],
                               [1,13],
                                [1,14],
                                [1,15],
                                [1,16],
                                
                                [4,13],
                               [4,14],
                               [4,15],
                               [4,16],
                               [3,13],
                                [3,14],
                                [3,15],
                                [3,16],
                                
                                [6,13],
                               [6,14],
                               [6,15],
                               [6,16],
                               [5,13],
                                [5,14],
                                [5,15],
                                [5,16],
                                
                                [8,13],
                               [8,14],
                               [8,15],
                               [8,16],
                               [7,13],
                                [7,14],
                                [7,15],
                                [7,16],
                                
                                [10,13],
                               [10,14],
                               [10,15],
                               [10,16],
                               [9,13],
                                [9,14],
                                [9,15],
                                [9,16],
                                
                                [12,13],
                               [12,14],
                               [12,15],
                               [12,16],
                               [11,13],
                                [11,14],
                                [11,15],
                                [11,16],
                               ],dtype=int) #corresponds to topics above

#topics_sensor = np.zeros(topics_node_to_node.shape,dtype=int)
#for i in xrange(topics_node_to_node.shape[0]):
#    topics_sensor[i] = (node_to_sensor[topics_node_to_node[i,0]],node_to_sensor[topics_node_to_node[i,1]])

#1,2,4,7,8,10
#node_indices = {1:0,2:1,13:2,14:3,15:4,16:5} #node index in distance vectors
bag_names = []
measured_distances = []
node_to_sensor = []

#bag_names.append("2015-08-27-13-25-56.bag")
#node_to_sensor.append({1:1,2:5,13:13,14:14,16:16})

bag_names.append("outdoor_ranging_2015-09-22-15-41-32.bag")

In [3]:
#process all bag files
plt.close('all')
bag_means = []
bag_std = []
bag_means_raw = []
bag_stds_raw = []
bag_means_nodes = []
bag_std_nodes = []
data_clean = []
for n_i,n in enumerate(bag_names):
    print n
    bag = rosbag.Bag(n)
    data = {}
    topics_mean = np.zeros(len(topics))
    topics_std = np.zeros(len(topics))
    plt.figure()
    d_supahclean = []
    for i,t in enumerate(topics):
        plt.subplot(4,12,i+1)
        data = []
        for m in bag.read_messages(topics=[t]):
            data.append((m[1].header.stamp.to_sec(),m[1].data))
        d = np.array(data)
        try:
            d_clean = d[np.where((d[:,1]>3.5)&(d[:,1]<20))[0]] #remove bad measurements
        except:
            d_supahclean.append(np.zeros((1,2)))
            print "err"
            continue
        if(d_clean.shape[0]>10):
            model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
            model_ransac.fit(np.arange(d_clean.shape[0]).reshape((-1,1)), d_clean)
            inlier_mask = model_ransac.inlier_mask_
            outlier_mask = np.logical_not(inlier_mask)
            m = d_clean[inlier_mask,1].mean()
            std = d_clean[inlier_mask,1].std()
            d_outlier = d_clean[np.where((np.abs(d_clean[:,1]-m)<3*std))[0],:]
            topics_mean[i] = d_outlier[:,1].mean()
            topics_std[i] = d_outlier[:,1].std()
            #plt.plot(d_clean)
            #plt.plot(d_clean[inlier_mask])
            #plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1],'.')
            #plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1])
            
            d = d_outlier.copy()
            good_data = []
            good_data.append(d[0])
            v_d = np.zeros(d.shape)
            for i in xrange(1,d.shape[0]):
                t_1 = good_data[-1][0]
                t = d[i,0]
                p_1 = good_data[-1][1]
                p = d[i,1]
                v = np.abs((p-p_1)/(t-t_1))
                v_d[i]=v
                if(v<1.5):
                    good_data.append(d[i])
            good_data=np.array(good_data)
            plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1],'+')
            plt.plot(good_data[:,0]-d_outlier[0,0],good_data[:,1],'.')
            plt.plot(good_data[:,0]-d_outlier[0,0],good_data[:,1],'-')
            d_supahclean.append(good_data.copy())
        else:
            topics_mean[i] = 0
            topics_std[i] = 0
            d_supahclean.append(np.zeros((1,2)))
    data_clean.append(d_supahclean)


outdoor_ranging_2015-09-22-15-41-32.bag


In [4]:
#convert to clean datasets (>3 measurements), fixed rate
meas = []
for n_i,n in enumerate(bag_names):
    data = data_clean[n_i]
    t0 = np.max([d[0,0] for d in data[:8]])
    tend = np.min([d[-1,0] for d in data[:8]])
    timesteps = np.arange(t0,tend,0.2)
    measurements = np.zeros((timesteps.shape[0],len(topics)))
    
    for t_i, t in enumerate(timesteps):
        for j in xrange(len(topics)):
            d = data[j]
            #get all matching data
            idx = np.where((d[:,0]>=t) & (d[:,0]<t+0.2))[0]
            if(idx.shape>0):
                measurements[t_i,j] = np.median(d[idx,1])
            else:
                measurements[t_i,j] = 0 
    meas.append(measurements[np.where(np.sum((measurements[:,:4]>1) & (measurements[:,:4]<20),1)>3)]) #how many valid measurements do we need?
    #TODO: for multiple nodes, make sure each one has at least 3 (or 4 is better) ranging measurements

/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [5]:
plt.figure()
for i in xrange(12):
    plt.subplot(3,4,i+1)
    plt.plot(meas[0][:,i*4:(i+1)*4])

In [292]:
res[-4:]
#array([ 4.13215675,  3.67599784,  4.18511672,  3.83472233])
#array([ 4.22836493,  3.80866668,  4.07504397,  3.75225553])
#array([ 4.18787562,  3.58978245,  4.12752976,  3.73947614]) #0.15 #[15,13,16,14]

array([ 4.18787562,  3.58978245,  4.12752976,  3.73947614])

In [293]:
print res[-o.shape[0]:].round(2)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p = res[:-dist_pairs.shape[0]].reshape((DOF,3))[:fixed_nodes.shape[0]]
p2 = res[:-dist_pairs.shape[0]].reshape((DOF,3))[fixed_nodes.shape[0]:]
ax.scatter(p[:,0],p[:,1],p[:,2],c='b')
for i in xrange(fixed_nodes.shape[0]):
    ax.text(p[i,0],p[i,1],p[i,2],fixed_nodes[i])

ax.scatter(p2[:,0],p2[:,1],p2[:,2],c='y')
#ax.scatter(pc[:,0],pc[:,1],pc[:,2],c='b')
#ax.scatter(p2c[:,0],p2c[:,1],p2c[:,2],c='r')

[ 4.19  3.59  4.13  3.74]


In [294]:
p
#array([[ 0.        ,  0.        ,  0.        ],
#       [-5.35      ,  0.        ,  0.        ],
#       [ 0.96      , -2.35      ,  0.        ],
#       [-2.71297727, -0.617745  , -2.3072566 ]])

array([[ 0.        ,  0.        ,  0.        ],
       [-5.35      ,  0.        ,  0.        ],
       [ 0.96      , -2.35      ,  0.        ],
       [-2.71297727, -0.617745  , -2.3072566 ]])

In [153]:
import scipy.spatial as ss
ss.distance.squareform(ss.distance.pdist(p))

array([[ 0.        ,  3.66324529,  3.30318086,  5.90031792],
       [ 3.66324529,  0.        ,  4.74821996,  3.60090027],
       [ 3.30318086,  4.74821996,  0.        ,  7.24703045],
       [ 5.90031792,  3.60090027,  7.24703045,  0.        ]])

In [295]:
print res[-o.shape[0]:].round(2)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for it in xrange(len(resall)):
    p = resall[it][:-dist_pairs.shape[0]].reshape((DOF,3))[:fixed_nodes.shape[0]]
    #p2 = res[:-dist_pairs.shape[0]].reshape((DOF,3))[fixed_nodes.shape[0]:]
    ax.scatter(p[:,0],p[:,1],p[:,2],c='b')
for i in xrange(fixed_nodes.shape[0]):
    ax.text(p[i,0],p[i,1],p[i,2],fixed_nodes[i])

#ax.scatter(p2[:,0],p2[:,1],p2[:,2],c='y')
#ax.scatter(pc[:,0],pc[:,1],pc[:,2],c='b')
#ax.scatter(p2c[:,0],p2c[:,1],p2c[:,2],c='r')

[ 4.19  3.59  4.13  3.74]


In [5]:
#create "variables"
def cost_function(x,y,m,o):
    '''
        Assumes x and y are Nx3, m and o are 1d vectors
    '''
    dist_xy = np.sqrt(np.sum((x-y)**2,1)) #1d vector
    cost_per_element = (dist_xy-(m-o))**2
    total_cost = cost_per_element.sum()
    return total_cost

def derivative_cost_function(x,y,m,o):
    '''
        Derivatives of the cost function wrt x,y and o
    '''
    dist_xy = np.sqrt(np.sum((x-y)**2,1)) #1d vector
    factor = 2*(dist_xy-(m-o))/dist_xy
    #derivative with respect to x
    deriv_x = factor.reshape((-1,1))*(x-y)
    #derivative with respect to y
    deriv_y = -deriv_x
    #derivative with respect to o
    deriv_o = factor*dist_xy
    return deriv_x, deriv_y, deriv_o
def create_data(fixed_nodes,floating_nodes,measurement_pairs,measurements):
    '''
        fixed nodes: indices in measurement_pairs corresponding to fixed nodes
        floating nodes: indices in measurement_paris correspond to floating nodes
        measurements pairs: each column in measurements corresponds to a distance between two nodes as specified by the rows in this matrix
        measurements: num_measurements x num pairs observed distances
        
    '''
    num_fixed = fixed_nodes.shape[0]
    num_floating = floating_nodes.shape[0]
    num_measurements = measurements.shape[0]
    measurement_pairs = measurement_pairs.copy()
    
    floating_to_fixed = {} #contains an array of indices in measurement_pairs for each node
    floating_to_floating = {} #similar
    bars = {1:(2,1.37),3:(4,1.37),5:(6,1.37),7:(8,1.37),9:(10,1.37),11:(12,1.37)} #nodes with bar connections: FROM: (TO,LENGTH)
    
    #for each floating node, find the measurement pairs to fixed nodes
    for node_idx, node in enumerate(floating_nodes):
        #get measurement columns corresponding to this node (to/from)        
        node_dist_idx = np.where((measurement_pairs[:,0] == node) | (measurement_pairs[:,1] == node))[0]
        #create new array
        floating_to_fixed[node] = []        
        for i in node_dist_idx:
            if(measurement_pairs[i,0] in fixed_nodes or measurement_pairs[i,1] in fixed_nodes):
                #found one
                floating_to_fixed[node].append(i)
                
    #for each floating node, find the measurement pairs to other floating nodes
    for node_idx, node in enumerate(floating_nodes):
        #get measurement columns corresponding to this node (to/from)        
        node_dist_idx = np.where((measurement_pairs[:,0] == node) | (measurement_pairs[:,1] == node))[0]
        #create new array
        floating_to_floating[node] = []        
        for pair_idx in node_dist_idx:
            #to prevent doubles, we only add pairs originating FROM this node TO another floating node
            if((measurement_pairs[pair_idx,0] == node) and measurement_pairs[pair_idx,1] in floating_nodes):
                #found one
                floating_to_floating[node].append(pair_idx)
    
    
    #Find all the floating node positions with enough measurements for ranging
    #enough means more than 3 valid measurements to fixed nodes
    floating_valid_points = np.zeros((num_measurements,num_floating),dtype=bool)
    measurements_valid = (measurements>1) & (measurements<20) #this matrix contains a one for each valid measurements
    #a one in the above matrix indicates that there are enough valid measurements between a floating node and fixed nodes available for the 
    #columns correspond to indices in floating_nodes
    for node_idx, node in enumerate(floating_nodes):
        for timestep_idx in xrange(num_measurements):
            #get all measurements to fixed nodes from this node
            to_fixed = floating_to_fixed[node]
            valid_cnt = 0
            for pair_idx in to_fixed:
                if(measurements_valid[timestep_idx,pair_idx]):
                    #this is a valid measurement, use it
                    valid_cnt += 1
            if(valid_cnt >= 4):
                #we can estimate this location at this timestep
                floating_valid_points[timestep_idx,node_idx] = True

    #we now know all the valid coordinates to estimate, so we are ready to create constraints
        
    #print floating_to_fixed
    #print floating_to_floating
        
    #create distance constraints between floating nodes and fixed nodes for all valid time steps with valid measurements per floating node
    constraints = []
    
    for timestep_idx in xrange(num_measurements): #iterate over time steps
        for node_idx, node in enumerate(floating_nodes): #iterate over nodes
            if(floating_valid_points[timestep_idx,node_idx]):
                #This is a valid measurement timestep, 
                #add a constraint for each valid measurement to the respective anchor.
                to_fixed = floating_to_fixed[node]
                for pair_idx in to_fixed:
                    if(measurements_valid[timestep_idx,pair_idx]):
                        constraints.append(("float_to_fixed",
                                            measurement_pairs[pair_idx],
                                            measurements[timestep_idx,pair_idx],
                                            timestep_idx))
                #add constraints to other valid floating nodes
                to_floating = floating_to_floating[node]
                for pair_idx in to_floating:
                    if(measurements_valid[timestep_idx,pair_idx]):
                        pair = measurement_pairs[pair_idx]
                        from_ = np.where(floating_nodes == pair[0])[0][0]
                        to_ = np.where(floating_nodes == pair[1])[0][0]
                        if(floating_valid_points[timestep_idx,from_] and floating_valid_points[timestep_idx,to_]): #make sure destination node is valid at the current timestep
                            
                            constraints.append(("float_to_float",
                                            measurement_pairs[pair_idx],
                                            measurements[timestep_idx,pair_idx],
                                            timestep_idx))
                        
                #add bar constraints
                if(node in bars):
                    b = bars[node]
                    bar_node = b[0]
                    #check whether destination node location is valid at this timestep
                    if(floating_valid_points[timestep_idx,np.where(floating_nodes == bar_node)[0][0]]):
                        #ok, we can add the constraint
                        constraints.append(("bar",
                                            [node,bar_node],
                                            b[1],
                                           timestep_idx))
    
    #create variables (this is the fun part)
    variable_to_node = [] #indicates which variable corresponds to which node
    for n in fixed_nodes:
        variable_to_node.append(n) #first indices are fixed nodes
    for timestep_idx in xrange(num_measurements): #iterate over time steps
        for node_idx, node in enumerate(floating_nodes): #iterate over nodes
            if(floating_valid_points[timestep_idx,node_idx]):
                variable_to_node.append((node,timestep_idx)) #we store the floating nodes as node, timestep tuples
    variable_to_node_map = {}
    for v_idx,v in enumerate(variable_to_node):
        variable_to_node_map[v] = v_idx
    
    #print len(variable_to_node)
    
    offset_variable_to_pair = [] #offset variables: pairs of nodes == tricky
    for c in constraints:
        #find out if we already have an offset variable for this constraint pair
        if(not c[0]=="bar"): #bar constraints don't have offsets
            pair = tuple(np.sort(c[1]))
            if(not pair in offset_variable_to_pair):
                offset_variable_to_pair.append(pair)
    
    num_offsets = len(offset_variable_to_pair)
    num_variables = len(variable_to_node)
    num_constraints = len(constraints)
    
    #print len(offset_variable_to_pair)
    #TODO: constraints
    #print len(constraints)
    
    #Generate the measurement vector, variable to constraint matrices and offset to constraint matrices
    measurement_vector = np.zeros(num_constraints)
    variables_to_CSTR_X = np.zeros((num_constraints,num_variables))
    variables_to_CSTR_Y = np.zeros(variables_to_CSTR_X.shape)
    offsets_to_CSTR = np.zeros((num_constraints,num_offsets))
    #print floating_valid_points
    for c_idx,c in enumerate(constraints):
        measurement_vector[c_idx] = c[2]
        #find the correct offset variable
        if(c[0]=="bar"):
            #bar constraints don't have offsets
            offsets_to_CSTR[c_idx,:] = 0 
        else:
            pair = tuple(np.sort(c[1]))
            offset_idx = offset_variable_to_pair.index(pair)
            offsets_to_CSTR[c_idx,offset_idx] = 1 
        #find the correct nodal coordinate variables
        if(c[0]=="bar" or c[0]=="float_to_float"):
            #print c
            from_ = tuple((c[1][0],c[3])) #node,timestep
            to_ = tuple((c[1][1],c[3]))
            from_idx = variable_to_node_map[from_]
            to_idx = variable_to_node_map[to_]
            variables_to_CSTR_X[c_idx,from_idx] = 1
            variables_to_CSTR_Y[c_idx,to_idx] = 1
        else:
            #float to fixed
            if(c[1][0] in fixed_nodes):
                from_ = c[1][0]
                to_ = tuple((c[1][1],c[3]))
            else:
                from_ = tuple((c[1][0],c[3]))
                to_ = c[1][1]

            from_idx = variable_to_node_map[from_]
            to_idx = variable_to_node_map[to_]
            variables_to_CSTR_X[c_idx,from_idx] = 1
            variables_to_CSTR_Y[c_idx,to_idx] = 1
        #variables_to_CSTR_X[c_idx,variable_to_node.index(c[1][0])] = 1
        #variables_to_CSTR_Y[c_idx,variable_to_node.index(c[1][1])] = 1
        
        
    def deriv_func(N,o):
        X = variables_to_CSTR_X.dot(N)
        Y = variables_to_CSTR_Y.dot(N)
        O = offsets_to_CSTR.dot(o.reshape((num_offsets,1))) 
        #print X.shape
        #print Y.shape
        #print O.shape
        #print m.shape
        dX,dY,dO = derivative_cost_function(X,Y,measurement_vector.ravel(),O.ravel())
        dN = variables_to_CSTR_X.T.dot(dX) +  variables_to_CSTR_Y.T.dot(dY)
        dN[0,:] = 0
        dN[1,1:] = 0
        dN[2,2] = 0
        do = offsets_to_CSTR.T.dot(dO.reshape((-1,1))).ravel()
        return dN,do
    
    def cost_func(N,o):
        X = variables_to_CSTR_X.dot(N)
        Y = variables_to_CSTR_Y.dot(N)
        O = offsets_to_CSTR.dot(o.reshape((num_offsets,1))) 
        return cost_function(X,Y,measurement_vector.ravel(),O.ravel())
    
    #print offsets_to_CSTR
    #print variables_to_CSTR_X
    #create m vector
    return deriv_func,cost_func, num_variables, num_offsets, variable_to_node, variable_to_node_map, offset_variable_to_pair

In [28]:
import scipy.io as sio
sio.savemat("data_outdoors_preprocessed.mat",
            {'fixed_nodes':fixed_nodes,'floating_nodes':floating_nodes,'dist_pairs':dist_pairs,'m':m})

In [23]:
cf = 10
cf_min = cf
results_and_costs = []
while cf>0.1:
    fixed_nodes = np.array([16,15,14,13],dtype=int)
    floating_nodes = np.array([1,2,5,6,7,8,9,10,11,12],dtype=int)#,2],dtype=int)
    #dist_pairs = np.array([[2,13],[2,14],[2,15],[2,16],
    #                       [1,13],[1,14],[1,15],[1,16],
    #                       [4,13],[1,14],[1,15],[1,16],
    #                       [6,13],[1,14],[1,15],[1,16],
    #                       [8,13],[1,14],[1,15],[1,16],
    #                       [,13],[1,14],[1,15],[1,16],
    #                       [1,13],[1,14],[1,15],[1,16]
    #                      ],dtype=int)
                           #np.array([[1,10],[2,10],[4,10],[7,10],[8,10],[12,10],[13,10],[16,10]],dtype=int)
    dist_pairs = topics_node_to_node
    
    #0,1,2,4,6,8,10

    a = np.arange(meas[0].shape[0])
    np.random.shuffle(a)
    m = meas[0][a[::7],:]#[:,(0,1,2,3,4,5,6,8)]
    print m.shape
    #m = np.hstack((m,np.ones((m.shape[0],1))*1.36))
    df,cf,DOF,num_offsets,variable_to_node,variable_to_node_map, offset_variable_to_pair  = 
    create_data(fixed_nodes,floating_nodes,dist_pairs,m)

    N,o = np.random.randn(DOF,3)*3,np.zeros(num_offsets)+3.8
    N[0] = 0
    N[1,1:] = 0
    N[2,2] = 0     
    #B=np.array([[ 0.,          0.,          0.        ],
    #                 [ -3.38,  0.,          0.        ],
    #                 [ -0.7039,  -2.189,  0.        ]])
    B = np.array([[0,0,0],
                 [9.97,0,0],
                 [3.15,11.16,0]])
    #N[:3] = B

    dN,do = df(N,o)
    #print cf(N,o)

    import scipy.optimize as so
    Nf = N.ravel()
    X_initial = np.hstack((Nf,o))
    def cost_fun(x):
        Nt = x[:-o.shape[0]].reshape((DOF,3))
        #Nt[:3] = B
        reg = 0.05*np.sum((x[-o.shape[0]:]-3.8)**2)#+0.5*(x[11]-2.5)**2#+0.05*(x[10]-2.)**2+0.05*(x[9]-2.)**2
        return cf(Nt ,x[-o.shape[0]:])+reg
    def cost_grad(x):
        dN,do = df(x[:-o.shape[0]].reshape((DOF,3)),x[-o.shape[0]:])
        #dN[:3] = 0
        #do[:] = 0
        do += 0.05*2*(x[-o.shape[0]:]-3.8)
        dN = dN.ravel()
        #dN[11] += 0.5*2*(x[11]-2.5)
        #dN[10] += 0.05*2*(x[10]-2.)
        #dN[9] += 0.05*2*(x[9]-2.)
        return np.hstack((dN,do))


    resall = []
    def progress(x):
        resall.append(x)

    res = so.fmin_bfgs(cost_fun,X_initial,fprime=cost_grad,disp=0,gtol=1e-10,callback=progress)
    cf = cost_fun(res)
    results_and_costs.append((res.copy(),cf,variable_to_node_map,offset_variable_to_pair,a.copy()))
    if(cf<cf_min):
        res_min = res.copy()
        DOF_min = DOF
        variable_to_node_map_min =  variable_to_node_map
        offset_variable_to_pair_min = offset_variable_to_pair
        cf_min = cf
    print cf

(93, 48)
7.67978883168
(93, 48)
59.3674071168
(93, 48)
9.49186303424
(93, 48)
16.8894457278
(93, 48)
53.7911434061
(93, 48)
10.9263155717
(93, 48)
11.9774879487
(93, 48)


KeyboardInterrupt: 

In [22]:

%prun so.fmin_bfgs(cost_fun,X_initial,fprime=cost_grad,disp=0,gtol=1e-10,callback=progress)

In [24]:
#cost_fun(res_min)
res=res_min.copy()

In [25]:
print res[-o.shape[0]:].round(2)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p = res[:fixed_nodes.shape[0]*3].reshape((fixed_nodes.shape[0],3))[:fixed_nodes.shape[0]]
#p[0,1]+=1
p2 = res[:-len(offset_variable_to_pair)].reshape(((res.shape[0]-len(offset_variable_to_pair))/3,3))[fixed_nodes.shape[0]:]
ax.scatter(p[:,0],p[:,1],p[:,2],c='b')
for i in xrange(fixed_nodes.shape[0]):
    ax.text(p[i,0],p[i,1],p[i,2],fixed_nodes[i])

ax.scatter(p2[:,0],p2[:,1],p2[:,2],c='y')
#ax.scatter(pc[:,0],pc[:,1],pc[:,2],c='b')
#ax.scatter(p2c[:,0],p2c[:,1],p2c[:,2],c='r')
ax.set_aspect('equal')

[ 4.06  3.76  4.03  3.81  3.62  3.76  3.59  4.22  4.    3.75  4.06  3.71
  4.11  3.68  4.06  3.8   3.91  3.87  3.92  3.85  4.04  3.68  4.02  3.75
  4.12  3.82  3.84  3.68  3.88  3.69  4.22  3.91  4.16  3.88  4.17  3.91
  4.37  3.58  4.04  3.75]


In [16]:
p_all = res[:12].reshape((4,3))
print p_all
print B
#np.linalg.norm(p_all[20]-p_all[21])

[[  0.           0.           0.        ]
 [-10.37797144   0.           0.        ]
 [ -3.26848811  11.53203134   0.        ]
 [ -7.15833895   4.28769715   0.78079217]]
[[  0.     0.     0.  ]
 [  9.97   0.     0.  ]
 [  3.15  11.16   0.  ]]


In [10]:
res[-10:]

array([ 3.82447262,  3.87178301,  3.65002531,  3.96786277,  4.00467714,
        3.81267492,  4.00032556,  3.86155599,  3.67771007,  3.97125125])

In [22]:
res[-o.shape[0]:]

array([ 3.94104845,  3.91390719,  3.98168363,  3.95158926,  3.91548744,
        3.92860805,  4.06065676,  3.97498433,  3.81817403,  3.94787904,
        4.00990517,  3.81977139,  3.84350277,  3.84733697,  4.04858242,
        3.94446129,  3.44145357,  4.31821431,  3.72509005,  3.99935807,
        3.89396468,  3.92934977,  4.03061451,  3.96842817,  3.79052255,
        3.94863262,  4.28768486,  4.05678094,  3.98560302,  3.86303144,
        3.93073998,  3.8429706 ,  4.4024697 ,  3.27055469,  3.95555874,
        3.75330747,  3.8570016 ,  3.74400632,  3.72075997,  4.2030231 ])

In [27]:
offset_variable_to_pair_min

[(2, 13),
 (2, 14),
 (2, 15),
 (2, 16),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (4, 13),
 (4, 14),
 (4, 15),
 (4, 16),
 (6, 13),
 (6, 14),
 (6, 15),
 (6, 16),
 (10, 13),
 (10, 14),
 (10, 15),
 (10, 16),
 (8, 13),
 (8, 14),
 (8, 15),
 (8, 16),
 (12, 13),
 (12, 14),
 (12, 15),
 (12, 16)]

In [28]:
res[-len(offset_variable_to_pair_min):]


array([ 4.16799159,  3.63648985,  4.03304079,  3.83795675,  4.02615586,
        3.68629254,  4.03762013,  3.89848137,  4.18677463,  3.62661513,
        4.02919902,  3.79128936,  3.24252253,  3.74993813,  3.54417684,
        3.63720535,  4.21208742,  3.54548237,  4.18295844,  3.71217001,
        4.06504594,  3.65841194,  4.2756098 ,  3.93173312,  3.3867485 ,
        3.76147812,  3.18438827,  3.92609695])

In [12]:
import scipy.io as sio
sio.savemat("offsets_outdoors_nobasepoints_given_longrun.mat",{"offset_variable_to_pair":np.array(offset_variable_to_pair_min),'offsets':res_min[-len(offset_variable_to_pair_min):]})
